# Loading

Create a generator function for importing the file

In [2]:
import urllib.request
import lzma
import base64
import re

def get_dependencies():
    localfile, info = urllib.request.urlretrieve("https://github.com/ogirardot/meta-deps/raw/master/pypi-deps.csv.lzma")
    with lzma.open(localfile, mode='r') as file:
        for line in file:
            package, version, deps = line.decode("utf-8").split('\t')
            deps = base64.b64decode(deps).decode("utf-8")
            deps = eval(deps)
            regex = r'\"*([\w\.]+)[><]?\=?.*\"*'
            
            def cleanDep(s):
                try:
                    match = re.search(regex, s)
                    match = re.search(regex, s)
                    s = match.group(0)
                    s = s.replace('"', '')
                    s = re.split(r'<>=', s)[0]
                    return s
                except:
                    return None
            
            deps = [cleanDep(x) for x in deps if cleanDep(x) is not None]
            
            yield package, deps
        
        


Store as dataframe

In [3]:
import pandas as pd
import numpy as np

rows = np.array([[None,None]])
for package, deps in get_dependencies():
    # Packages with no dependencies get blank string
    if deps:
        args= [[package, dep] for dep in deps]
    else:
        args = [[package, None]]
    rows = np.append(rows, args, axis=0)

        
df = pd.DataFrame(rows, columns=["package", "dependency"]).drop_duplicates()
df.head()

,package,dependency
0,None,None
1,0x10c-asm,None
2,2gis,None
3,3to2,None
4,3to2_py3k,None


In [4]:
deps = pd.Series(df.dependency.unique())
packages = pd.Series(df.package.unique())
new_deps = deps[~deps.isin(packages)]
new_df = pd.DataFrame({'package': new_deps, 'dependency': [None]*len(new_deps)})
df = pd.concat([df, new_df])
df = df.dropna(subset=['package'])

In [7]:
from timeit import default_timer, timeit

## Redshift

In Redshift we create a single table that will hold one "edge" per row.

In [2]:
import redshift_connector
conn = redshift_connector.connect(
     host='redshift-python-dep.cantgzv41ter.us-east-1.redshift.amazonaws.com',
     database='deps',
     user='awsuser',
     password='uGdipKs9WpJ8yU6'
  )
conn.autocommit = True
cursor = conn.cursor()

Create the table

In [9]:
cursor.execute("DROP TABLE dependencies")
cursor.execute(
"""
    CREATE TABLE dependencies (
        package VARCHAR(150),
        dependency VARCHAR(150) NULL
    )
"""
)

Load the data

In [13]:
from itertools import product

# Time it
start = default_timer()

#cursor.write_dataframe(df, "dependencies")
cursor.fastexecute = True
cursor.executemany("INSERT INTO dependencies VALUES (%s, %s)", df.itertuples(index=False, name=None))

end = default_timer()
redshift_load_time = end - start
redshift_load_time

4195.223583109

# Neptune Graph

Clear existing data from graph

In [14]:
%%gremlin

g.V().drop()

Load in nodes and edges

In [15]:
from gremlin_python import statics
from gremlin_python.structure.graph import Graph
from gremlin_python.process.graph_traversal import __
from gremlin_python.process.strategies import *
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection
from gremlin_python.process.traversal import T
from gremlin_python.process.traversal import Order
from gremlin_python.process.traversal import Scope

In [16]:
def load_nodes(g, df):
    for package_name in df.package.unique():
        try:
            g.add_v('package').property(T.id, package_name).as_(package_name).next()
            #print("added package: " + package_name)
        except:
            print(f"Failed to add package {package_name}")

def load_edges(g, df):
    for index, row in df.iterrows():
        if row['dependency'] is not None:
            g.V(row['package']).add_e("depends_on").to(__.V(row['dependency'])).next()
            #print("Added dependency: " + row['package'] + "->" + row['dependency'])



In [17]:
remoteConnStr = 'wss://data516.cluster-crrfpo5iyyda.us-east-1.neptune.amazonaws.com:8182/gremlin'
remoteConn = DriverRemoteConnection(remoteConnStr,'g')

g = Graph().traversal().withRemote(remoteConn)

start = default_timer()

load_nodes(g, df)
load_edges(g, df)

end = default_timer()
remoteConn.close()

neptune_load_time = end - start
neptune_load_time

163.17469938799968

In [18]:
%%gremlin
g.V()
g.E()

# Queries

## Helper Funcs

In [5]:
def timeRedshiftQuery(sqlTxt):
    cursor = conn.cursor()
    start = default_timer()
    cursor.execute(sqlTxt)
    end = default_timer()
    return (end - start) * 1000

def timeNeptuneQuery(f):
    start = default_timer()
    g = Graph().traversal().withRemote(remoteConn)
    end = default_timer()
    return end - start

## Redshift Recursive View
We save a view that has a row for each package and package it depends on either directly or indirectly.

## 1: Which package has the most direct dependencies and how many?

### Redshift

In [8]:
cursor = conn.cursor()
query = """
    SELECT TOP 1 *
    FROM 
    (
        SELECT DISTINCT package, count(*) as "dep_count"
        FROM dependencies
        WHERE dependency is not NULL
        GROUP BY package
    ) DepCounts
    ORDER BY dep_count DESC, package ASC

"""
cursor.execute(query)
cursor.fetchall()


(['Products.CMFPlone', 100],)

In [11]:
timeRedshiftQuery(query)

5.465979999996762

### Neptune

In [14]:
%%gremlin

g.V().hasLabel('package').in('depends_on').
groupCount().by(id).
order(Scope.local).
    by(values, Order.desc).
    by(keys, Order.asc).
unfold().limit(1)

# 2: Which package has the most total dependencies?


## Redshift

In [17]:
query = """
WITH RECURSIVE total_deps(package, dependency, level) AS 
    (SELECT package, dependency, 0 AS level 
     FROM dependencies
     UNION ALL
     SELECT d.package, d.dependency, level + 1
     FROM dependencies d, total_deps p
     WHERE 
        d.dependency = p.package AND
        d.dependency is not NULL AND
        level <= 50
    )
SELECT TOP 1 package, dep_count from
(
    SELECT package, count(dependency) as dep_count
    FROM total_deps
    GROUP BY package, dependency
) t
ORDER BY dep_count DESC, package ASC
"""
cursor.execute(query)
cursor.fetchall()

(['horae.planning', 130303],)

In [16]:
timeRedshiftQuery(query)

148.2574909999812

## Neptune

In [18]:
%%gremlin

g.V().hasLabel('package').emit().repeat(__.in('depends_on')).until(
    inE().count().is(0).or(
    loops().count().is(15))
).
groupCount().by(id).
order(Scope.local).
    by(values, Order.desc).
    by(keys, Order.asc).
unfold().limit(1)

# 3: Which packages depend on Pandas (directly and indirectly)?

## Redshift

In [26]:
query = """
    WITH RECURSIVE panda_deps(package, dependency, level) AS 
    (SELECT package, dependency, 0 AS level 
     FROM dependencies
     WHERE dependency = 'numpy'
     UNION ALL
     SELECT d.package, d.dependency, level + 1
     FROM dependencies d, panda_deps p
     WHERE d.dependency = p.package
    )
    SELECT package FROM panda_deps;

"""
cursor.execute(query)
cursor.fetchall()

(['BiologicalProcessNetworks'],
 ['gyroid'],
 ['lmj.nethack'],
 ['nplook'],
 ['oceans'],
 ['pyasa'],
 ['PyBUFR'],
 ['pyorbital'],
 ['python-consume'],
 ['pyucsc'],
 ['RTM'],
 ['smith'],
 ['bw2data'],
 ['canvas'],
 ['chebpy'],
 ['daft'],
 ['emcee'],
 ['fitsio'],
 ['gceising'],
 ['gsw'],
 ['h5dj'],
 ['linguistic-helper-functions'],
 ['lmj.particle'],
 ['lmj.pursuit'],
 ['mlboost'],
 ['mofa'],
 ['nwalign'],
 ['ocupy'],
 ['pyccv'],
 ['pyssp'],
 ['topzootools'],
 ['Bravo'],
 ['colorcorrect'],
 ['his2h5'],
 ['lmj.kohonen'],
 ['lmj.plot'],
 ['metaseq'],
 ['multichain_mcmc'],
 ['pyflycapture2'],
 ['pyRserve'],
 ['readfmf'],
 ['SimpleHist'],
 ['Spear'],
 ['ADAM-Tools'],
 ['dcmt'],
 ['django-instakit'],
 ['fastinterval'],
 ['hdf5-django'],
 ['lmj.c3d'],
 ['lmj.rbm'],
 ['patsy'],
 ['zipline'],
 ['pyucsc'])

In [27]:
timeRedshiftQuery(query)

0.004986631000065245

# 4: Which packages have multiple dependency?


## Redshift

In [37]:
query = """
WITH RECURSIVE total_deps(package, dependency, level) AS 
    (SELECT package, dependency, 0 AS level 
     FROM dependencies
     UNION ALL
     SELECT d.package, d.dependency, level + 1
     FROM dependencies d, total_deps p
     WHERE 
        d.dependency = p.package AND
        d.dependency is NOT NULL AND
        level <= 50
    )
SELECT DISTINCT package from
(
    SELECT package, count(dependency) as total_paths
    FROM total_deps
    GROUP BY package, dependency
    HAVING total_paths > 1
) t
"""
cursor.execute(query)
cursor.fetchall()

(['affinitic.verifyinterface'],
 ['anyvc'],
 ['archetypes.fieldtraverser'],
 ['atreal.massloader'],
 ['Attest'],
 ['baas'],
 ['beanstalk-stack'],
 ['blanc-basic-podcast'],
 ['bowerrecipe'],
 ['carp'],
 ['celery-tracker'],
 ['celery-with-redis'],
 ['chill'],
 ['ckanrdf'],
 ['clearwind.arecibo'],
 ['cloudplay'],
 ['clwapp'],
 ['cmsplugin-blog'],
 ['codespeed'],
 ['collective.atimage.transformmenu'],
 ['collective.baseid'],
 ['collective.blog.star'],
 ['collective.blueprint.base'],
 ['collective.contentstats'],
 ['collective.folderishtypes'],
 ['collective.geo.opensearch'],
 ['collective.googlevisualization'],
 ['collective.gtags'],
 ['collective.jqueryuithememanager'],
 ['collective.nextprev'],
 ['collective.opensearch'],
 ['collective.pfg.soup'],
 ['collective.portlet.references'],
 ['collective.redirect'],
 ['collective.sugarcrm'],
 ['collective.templateengines'],
 ['compoze'],
 ['cornerstone.soup'],
 ['cursedmongo'],
 ['dataflake.ldapconnection'],
 ['django-absolute'],
 ['django-audit

In [40]:
timeRedshiftQuery(query)

0.022346106999975746

## Neptune

From each node, traverse out. Store visited nodes, and return if a visited node is found.

In [41]:
%%gremlin
g.V().as('a').
    repeat(aggregate('visited').out()).until(out().as('visited')).
    select('a').dedup()